In [ ]:
from rhs_slm import SLM
from scipy.integrate import solve_ivp
from radauDAE import RadauDAE
from plot import plot_distributions_slm
from total_current import total_current_slm

import matplotlib.pyplot as plt
import parameters, grid, initial_conditions, mass_slm, jacobian_slm
import time

start_time = time.time()
params = parameters.Params()
grid = grid.Grid(params)

ic = initial_conditions.Initial_Conditions(params, grid)
mass = mass_slm.Mass(params, grid)
jac = jacobian_slm.Jac(params)
sol_init = ic.sol_init_slm
mass = mass.M
jac = jac.jac

tf = 1
method = RadauDAE
rtol = params.rtol
atol = params.atol
dae_fun = lambda t, u: SLM(t, u, ’pbi’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=False, mass=mass)

"""
dae_fun = lambda t, u: SLM(t, u, ’precondition’)
sol_init = sol.y[:,-1]
sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf), y0=sol_init,
rtol=rtol, atol=atol, jac_sparsity=jac,
method=method, vectorized=False, dense_output=True,
100
mass=mass)
"""

sol_init = sol.y[:,-1]
dae_fun = lambda t, u: SLM(t, u, ’test1’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, 1), y0=sol_init,
rtol=rtol, atol=atol, jac_sparsity=jac,
method=method, vectorized=False, dense_output=True,
mass=mass)
t = sol.t
u = sol.y
plot_distributions_slm(grid.x, u)
J = total_current_slm(t, u)

plt.figure(5)
plt.plot(t, J[0])
plt.xlim([0,5*10e-7])
plt.ylim([0,1.5])
plt.xlabel(’Time’)
plt.ylabel(’Current␣Density’)
plt.title("Ref␣Figure␣5")
n = u[2*params.N+2:3*params.N+3,-1]
plt.figure(6)
plt.plot(grid.x, n)
plt.xlim([0,1])
plt.ylim([0,0.06])
plt.xlabel(’Distance,␣x’)
plt.ylabel(’Electron␣Concentration,␣n’)
plt.title("Ref␣Figure␣6")
print("---␣%s␣seconds␣---" % (time.time() - start_time))